# Cipher Decryption

## Procedure
![Procedure](./procedure.png)

In [1]:
# lots of imports!
import string
import typing as t
from collections import defaultdict
from functools import partial, reduce

import nltk
import numpy as np
from nltk import word_tokenize

if t.TYPE_CHECKING:
    import typing_extensions as te

# typevar stuff
T = t.TypeVar('T')
T_contra = t.TypeVar('T_contra', contravariant=True)

class SupportsLessThan(t.Protocol[T_contra]):
    def __lt__(self, __other: T_contra) -> bool: ...

SupportsLessThanT = t.TypeVar('SupportsLessThanT', bound=SupportsLessThan)

In [2]:
# cipher generator, creates a substitution cipher
class SubstitutionCipherGenerator:
    def __init__(self, seed: t.Optional[int] = None):
        self.random = np.random.default_rng(seed)

    def generate(self) -> str:
        return ''.join(string.ascii_uppercase[x] for x in self.random.permutation(26))

    def __iter__(self) -> 'te.Self':
        return self

    def __next__(self) -> str:
        return self.generate()

    @staticmethod
    def to_translate(cipher: str) -> t.Dict[int, t.Optional[int]]:
        return str.maketrans(string.ascii_uppercase, cipher)

In [3]:
cipher_gen = SubstitutionCipherGenerator(1234)
next(cipher_gen)

'VJNKYFDZICRAQLPMWHGESOUXBT'

In [4]:
# download and read Mobi Dick as corpus (UTF-8 BOM)
!wget -O files/moby_dick.txt -nc https://lazyprogrammer.me/course_files/moby_dick.txt
corpus = word_tokenize(open('files/moby_dick.txt', encoding='utf-8-sig').read().upper())
corpus[:10]

File ‘files/moby_dick.txt’ already there; not retrieving.


['CHAPTER', '1', '.', 'LOOMINGS', '.', 'CALL', 'ME', 'ISHMAEL', '.', 'SOME']

In [5]:
# build a character-based language model (1- and 2-grams)
class LanguageModel:
    remove_punctuation = str.maketrans('', '', string.punctuation)
    vocab = defaultdict(lambda: 26, zip(string.ascii_uppercase, range(26)))

    def __init__(self):
        self.pi: np.ndarray = np.ones(27)
        self.A: np.ndarray = np.ones((27, 27))
        self.P: np.ndarray = np.ones(27)

    def fit(self, documents: t.List[str]) -> 'te.Self':
        for document in documents:
            document = document.upper().translate(self.remove_punctuation)
            if not document:
                continue
            generator = iter(document)
            first = next(generator)
            last = self.vocab[first]
            self.pi[last] += 1
            self.P[last] += 1
            for character in generator:
                idx = self.vocab[character]
                self.A[last, idx] += 1
                self.P[idx] += 1
                last = idx
        self.pi = np.log(self.pi / (np.sum(self.pi) + 27))
        self.A = np.log(self.A / (np.array([np.sum(self.A, axis=1)]).T + 26))
        self.P = np.log(self.P / (np.sum(self.P) + 27))
        return self

    def probability(self, document: str) -> np.floating:
        tokens = word_tokenize(document)
        result = np.float64(0)
        for token in tokens:
            token = token.upper().translate(self.remove_punctuation)
            if not token:
                continue
            generator = iter(token)
            last = self.vocab[next(generator)]
            result += self.pi[last] + self.P[last]
            for character in generator:
                idx = self.vocab[character]
                result += self.A[last, idx] + self.P[idx]
                last = idx
        return result

In [6]:
model = LanguageModel().fit(corpus)
model.P

array([-2.51281796, -4.04021945, -3.7522212 , -3.22106444, -2.10497572,
       -3.83044031, -3.82970805, -2.7261953 , -2.6848464 , -6.7895887 ,
       -4.77808126, -3.11097836, -3.71856204, -2.68219406, -2.62829562,
       -4.01882603, -6.41471871, -2.91482374, -2.70491092, -2.38903189,
       -3.58200044, -4.71925156, -3.7689801 , -6.842767  , -4.04015922,
       -7.31974625, -4.77280421])

In [7]:
model.pi

array([ -2.23649354,  -3.00084596,  -3.27888866,  -3.64681714,
        -4.03782163,  -3.29291793,  -4.15306287,  -2.79040537,
        -2.72089777,  -5.58797166,  -5.4708722 ,  -3.61109286,
        -3.27596004,  -3.81755352,  -2.75425197,  -3.61024143,
        -5.73192442,  -4.05679453,  -2.41089207,  -1.83012394,
        -4.41777318,  -4.94075895,  -2.72650906, -10.4969433 ,
        -4.48445139,  -8.76234225,  -3.51966196])

In [8]:
model.A[model.vocab['A'], model.vocab['R']]

-2.223640267275985

In [9]:
model.probability('xyz')

-42.107382554844726

In [10]:
model.probability('are')

-13.239658835751746

In [11]:
# cipher encoder and decoder
class Cipher:
    def __init__(self, cipher: str):
        self.cipher = str.maketrans(string.ascii_uppercase, cipher)
        self.reverse = str.maketrans(cipher, string.ascii_uppercase)

    def encode(self, text: str) -> str:
        return text.upper().translate(self.cipher)

    def decode(self, text: str) -> str:
        return text.upper().translate(self.reverse)

In [12]:
# utility functions
def encode(text: str, cipher: str):
    return Cipher(cipher).encode(text)

def decode(text: str, cipher: str):
    return Cipher(cipher).decode(text)

In [13]:
# genetic algorithm
def genetic_algorithm(
    f: t.Callable[[T], SupportsLessThanT],
    rand: t.Callable[[], T],
    mutate: t.Callable[[T, int], t.List[T]],
    offspring: int = 3,
    parents: int = 5,
    iterations: int = 10000,
) -> t.List[T]:
    def mutator(parent: T) -> t.List[T]:
        return mutate(parent, offspring)
    pool = [rand() for _ in range(offspring * parents + parents)]
    for i in range(iterations):
        if i > 0:
            pool += reduce(list.__add__, map(mutator, pool))
        scores = list(map(f, pool))
        indices = sorted(range(len(pool)), key=scores.__getitem__, reverse=True)
        pool = list(map(pool.__getitem__, indices[:parents]))
    return pool

In [14]:
# create cipher text
raw_text = '''I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was compelled to listen to.'''
real_cipher = cipher_gen.generate()
cipher_text = encode(raw_text, real_cipher)
real_cipher, cipher_text[:10]

('JCDTMKYPUHRZQONWGEBXVAFLIS', 'U XPMO ZNV')

In [15]:
# prepare for the algorithm
def f(cipher: str) -> float:
    decoded = decode(cipher_text, cipher)
    return float(model.probability(decoded))

random_cipher = cipher_gen.generate

class CipherMutator:
    def __init__(self, seed: t.Optional[int] = None):
        self.random = np.random.default_rng(seed)

    def mutate(self, cipher: str) -> str:
        i = j = 0
        while i == j:
            i, j = self.random.choice(len(cipher), size=2)
        if i > j:
            i, j = j, i
        return cipher[:i] + cipher[j] + cipher[i+1:j] + cipher[i] + cipher[j+1:]

    def mutate_many(self, cipher: str, count: int) -> t.List[str]:
        ciphers = set()
        while len(ciphers) < count:
            ciphers.add(self.mutate(cipher))
        return list(ciphers)

mutator = CipherMutator(4321)
mutate = mutator.mutate_many

In [16]:
# run the algorithm!
solution = genetic_algorithm(f, random_cipher, mutate, iterations=1000)
solution, real_cipher

(['JCDTMKYPUHSZQONWGEBXVAFLIR',
  'JCDTMKYPUHSZQONWGEBXVAFLIR',
  'JCDTMKYPUHSZQONWREBXVAFLIG',
  'JCDTMKYPUGSZQONWHEBXVAFLIR',
  'JCDTMKYPUGSZQONWHEBXVAFLIR'],
 'JCDTMKYPUHRZQONWGEBXVAFLIS')

In [17]:
# what would that look like?
decode(cipher_text, solution[0])

'I THEN LOUNGED DOWN THE STREET AND FOUND,\nAS I EXPECTED, THAT THERE WAS A MEWS IN A LANE WHICH RUNS DOWN\nBY ONE WALL OF THE GARDEN. I LENT THE OSTLERS A HAND IN RUBBING\nDOWN THEIR HORSES, AND RECEIVED IN EXCHANGE TWOPENCE, A GLASS OF\nHALF-AND-HALF, TWO FILLS OF SHAG TOBACCO, AND AS MUCH INFORMATION\nAS I COULD DESIRE ABOUT MISS ADLER, TO SAY NOTHING OF HALF A DOKEN\nOTHER PEOPLE IN THE NEIGHBOURHOOD IN WHOM I WAS NOT IN THE LEAST\nINTERESTED, BUT WHOSE BIOGRAPHIES I WAS COMPELLED TO LISTEN TO.'

In [18]:
model.probability(raw_text), model.probability(decode(cipher_text, solution[0]))

(-2064.330329785218, -2058.3277878035306)

In [19]:
for i, (guess, real) in enumerate(zip(solution[0], real_cipher)):
    if guess != real:
        print(chr(ord('A') + i))

K
Z
